### Edgar Moises Hernandez-Gonzalez
#### 24/07/20-26-08-20
#### Clasificacion binaria de imagenes motoras en EEG con CNN-2D + LSTM
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)
##### Entrada resize proporcional de 67x500

In [1]:
from google.colab import drive

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import time
import pywt
import cv2
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from keras.models import load_model

In [4]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9E.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_test_9.csv", header=None)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(400, 3000)
(320, 3000)
(320, 1)


In [6]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 2


In [7]:
#unir 3 canales
def unir_escalogramas_vertical(data):
  dim = (500, 67) #ancho, alto
  fs = 250
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  datos = np.zeros((data.shape[0],67,500))
  temporal = np.zeros((135,1000))
  for i in range(data.shape[0]): #n muestras
    for j in range(3): #3 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      # escalograma CWT
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3', sampling_period = (1 / fs))
      temporal[j*45:(j+1)*45, :] = abs(coef)
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [8]:
#llamar a unir_escalogramas_vertical(data)
inicio = time.time()

x_train = unir_escalogramas_vertical(x_train)
x_test = unir_escalogramas_vertical(x_test)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
0
100
200
300
Tiempo: 38.170889377593994


In [9]:
print(x_test.shape)

(320, 67, 500)


In [10]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= np.ceil(np.max(x_train))

In [11]:
#convertir de 3D a 5D
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(320, 1, 67, 500, 1)


In [12]:
model = load_model('/content/drive/My Drive/BCI-IV-2b/Modelos/CWT_RNN_09.hdf5')

In [13]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 1, 67, 500, 8)     80        
_________________________________________________________________
time_distributed_12 (TimeDis (None, 1, 33, 250, 8)     0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 1, 33, 250, 8)     584       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 1, 16, 125, 8)     0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 1, 16000)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 8)                 512288    
_________________________________________________________________
dense_5 (Dense)              (None, 16)               

In [21]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 1.6913301944732666


In [16]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.4315
accuracy: 0.5281


In [17]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [18]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.6687 


In [19]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[138  22]
 [ 31 129]]


In [22]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,1,67,500,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.047023773193359375
